In [8]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")


In [9]:
#df = pd.read_csv('spotify_songs.csv')
df = pd.read_csv('spotify_millsongdata.csv')
#df.iloc[10000:10100]
#df['playlist_subgenre']

spotify_df = pd.read_csv('dataset.csv')
spotify_df.iloc[13:26]
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [10]:
#Dataset names
#tracks_features
#dataset
#data

spotify_df['popularity'].describe()

spotify_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [11]:
#bucketing popularity comlumn

float_cols= spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [12]:
ohe_cols = 'popularity'

spotify_df['popularity'].describe()

count    114000.000000
mean         33.238535
std          22.305078
min           0.000000
25%          17.000000
50%          35.000000
75%          50.000000
max         100.000000
Name: popularity, dtype: float64

In [13]:
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))
spotify_df.iloc[1522]

Unnamed: 0                                                       1522
track_id                                       0AAk1eycDCCsCe3AP7NKhq
artists             The New Mastersounds;Eddie Roberts;Lamar Willi...
album_name                                          The Deplar Effect
track_name                                                     Before
popularity                                                         21
duration_ms                                                    244560
explicit                                                        False
danceability                                                     0.58
energy                                                          0.549
key                                                                 4
loudness                                                      -12.382
mode                                                                0
speechiness                                                    0.0379
acousticness        

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def find_genre(track_id):
    '''
    finds genre of inputed track id by calling genres of artist of the song
    '''
    CLIENT_ID = "ef3a0b8293f84aae9cc15dda27c9b462"
    CLIENT_SECRET = "373610a9af5445d6925f21938ace6fc6"

    #track_id = "0AAk1eycDCCsCe3AP7NKhq"

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

    track_data = sp.track(track_id)

    artist_ids = []

    for artist in track_data["artists"]:
        artist_ids.append(artist["id"])

    artists_data = sp.artists(artist_ids)

    genres = []

    for artist in artists_data["artists"]:
        genres += artist["genres"]

    genres = set(genres) # removes duplicates

    return(list(genres))

find_genre('14kf1Cdj0sdtKtMY02IBcB')

['latin alternative']

In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
genre = []
dictionary = {}
CLIENT_ID = "d6fa070298f044288363a1c02438a3cf"
CLIENT_SECRET = "31146b976cd54a96bcc7d35ad194ff95"
counter = 0
import time
import tqdm

# I used this raw version for the large csv file as I could end it during runtime and more conveniently pull values
#however the function is the more practical/ correct version, and is used for the playlist later:

# for entry in tqdm.tqdm(spotify_df.iloc[0:114000]['track_id']):
#     track_id = entry

#     sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

#     try:
#         track_data = sp.track(track_id)

#         artist_ids = []

#         for artist in track_data["artists"]:
#             artist_ids.append(artist["id"])

#         artists_data = sp.artists(artist_ids)

#         genres = []

#         for artist in artists_data["artists"]:
#             genres += artist["genres"]

#         genres = set(genres) # removes duplicates

#         #print(list(genres))
#         #genre.append(dict(counter:genres))
#         dictionary[counter] = genres
#     except:
#         pass
    
#     counter +=1 
#     time.sleep(0.5)


def genre_generate(track_ids):
    '''
    generates genres of list of track_ids, similar to previous function.
    returns it as a dictinoary with index's as the key, this is so you can view if values were skipped during the process
    '''
    genre = []
    dictionary = {}
    counter = 0
    CLIENT_ID = "d6fa070298f044288363a1c02438a3cf"
    CLIENT_SECRET = "31146b976cd54a96bcc7d35ad194ff95"
    for entry in track_ids:
        track_id = entry

        sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

        try:
            track_data = sp.track(track_id)

            artist_ids = []

            for artist in track_data["artists"]:
                artist_ids.append(artist["id"])

            artists_data = sp.artists(artist_ids)

            genres = []

            for artist in artists_data["artists"]:
                genres += artist["genres"]

            genres = set(genres) # removes duplicates

            #print(list(genres))
            #genre.append(dict(counter:genres))
            dictionary[counter] = genres
        except:
            pass

        counter +=1 
        time.sleep(0.5)
    return dictionary


In [16]:
#this will return blank unless you run the commented out code above until the API throttles
#don't worry about this now, as I've written the results I got to a csv regardless
#the rest of the code just shows the process of how I did that, even if you can't follow it without running the large for loop
dictionary


{}

In [17]:
column_genre = []

for value in dictionary.values():
    column_genre.append(list(value))
    
column_genre
print(len(column_genre))

0


In [18]:
#spotify_df.iloc[0:2495]

spotify_reduced = spotify_df.iloc[0:2495]

spotify_reduced['genre'] = column_genre

spotify_reduced

ValueError: Length of values (0) does not match length of index (2495)

In [ ]:
temp = 0
count = 0
for key in dictionary:
    #print(key)
    if key!=temp:
        count +=1
    temp +=1
    
count

#no skipped values for first x_number values, this is our way to double check if our code had to skip a row or not

In [19]:
#write this reduced dataframe to a csv:
spotify_reduced.to_csv('genre_spotify.csv')

In [522]:
#confirm by reading this csv back:
spotify_reduced = pd.read_csv('genre_spotify.csv')
spotify_reduced.rename(columns={"track_id": "id"})

spotify_reduced

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,popularity_red,genre
0,0,0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.03220,0.000001,0.3580,0.7150,87.917,4,acoustic,14,"['j-pop', 'japanese singer-songwriter', 'j-aco..."
1,1,1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.92400,0.000006,0.1010,0.2670,77.489,4,acoustic,11,['acoustic chill']
2,2,2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.21000,0.000000,0.1170,0.1200,76.332,4,acoustic,11,"['lilith', 'uk pop', 'neo mellow', 'ectofolk',..."
3,3,3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.90500,0.000071,0.1320,0.1430,181.740,3,acoustic,14,"['viral pop', 'acoustic pop']"
4,4,4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.46900,0.000000,0.0829,0.1670,119.949,4,acoustic,16,"['singer-songwriter pop', 'acoustic pop']"
5,5,5,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.4810,6,-8.807,1,0.1050,0.28900,0.000000,0.1890,0.6660,98.017,4,acoustic,11,"['acoustic pop', 'neo mellow']"
6,6,6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,2,-8.822,1,0.0355,0.85700,0.000003,0.0913,0.0765,141.284,3,acoustic,14,"['neo mellow', 'dance pop', 'pop rock', 'piano..."
7,7,7,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,11,-9.331,1,0.0417,0.55900,0.000000,0.0973,0.7120,150.960,4,acoustic,16,"['acoustic pop', 'pop', 'dance pop', 'neo mell..."
8,8,8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,0,-8.700,1,0.0369,0.29400,0.000000,0.1510,0.6690,130.088,4,acoustic,14,"['acoustic pop', 'pop', 'dance pop', 'neo mell..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [523]:
#TFIDF
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

#ohe function pulled from github: https://github.com/madhavthaker/spotify-recommendation-system/blob/main/spotify-recommendation-engine.ipynb

In [20]:
#function to build entire feature set
#builds off of previous TFIDF Function
#following functions modified from same github listing, only kept aspects that are applicable to my datasets
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genre'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    #year_ohe = ohe_prep(df, 'year','year') * 0.5
    #popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled], axis = 1)

    #add song id
    final['track_id']=df['track_id'].values
    
    return final


#Had to make 2 seperate functions as one of my dataframes was having issues with the build in join function, 
#as it wasn't needed
def create_feature_set_no_lambda(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genre'])#.apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    #year_ohe = ohe_prep(df, 'year','year') * 0.5
    #popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled], axis = 1)

    #add song id
    final['track_id']=df['track_id'].values
    
    return final

In [21]:
#build weight table using previous functions
pd.set_option('display.max_columns', False)
pd.set_option('display.max_rows', False)
float_cols = spotify_reduced.dtypes[spotify_reduced.dtypes == 'float64'].index.values


complete_feature_set = create_feature_set_no_lambda(spotify_reduced, float_cols=float_cols)#.mean(axis = 0)

complete_feature_set


KeyError: 'genre'

In [22]:
#rename for future code to match names
playlist = pd.read_csv('My Spotify Library.csv')
playlist
#load my playlist
#downloaded a csv of playlist
playlist = playlist.rename(columns={"Spotify - id": "track_id"})

In [534]:
features = sp.audio_features(playlist['track_id'])

features_df = pd.DataFrame(data=features, columns=features[0].keys())

#we're going to trim this down to match our previous dataset:
#though we'll save an old version just in case:
features_df_backup = features_df
del features_df['type']
del features_df['duration_ms']
del features_df['track_href']
del features_df['uri']
del features_df['analysis_url']
del features_df['time_signature']
features_df['track_id'] = features_df['id']
del features_df['id']
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id
0,0.872,0.723,1,-5.968,1,0.2470,0.02930,0.000000,0.327,0.843,93.279,4VQNCzfZ3MdHEwwErNXpBo
1,0.800,0.585,10,-7.343,1,0.0924,0.26400,0.000000,0.153,0.779,126.058,6PGoSes0D9eUDeeAafB2As
2,0.600,0.537,9,-7.777,0,0.0362,0.04680,0.003800,0.206,0.126,90.061,6Ec5LeRzkisa5KJtwLfOoW
3,0.743,0.344,1,-11.174,1,0.1180,0.16900,0.008790,0.103,0.577,160.150,0wXuerDYiBnERgIpbb3JBR
4,0.775,0.736,1,-8.072,0,0.2470,0.31900,0.001160,0.127,0.362,100.988,2g6tReTlM2Akp41g0HaeXN
5,0.621,0.627,7,-7.762,0,0.1360,0.20900,0.000131,0.667,0.283,71.053,0GSbjG7c0gm7qX0Ru7vuon
6,0.403,0.646,0,-8.190,0,0.0372,0.44300,0.000557,0.131,0.418,93.981,0qKX14YZHptDWiEN0CgxGz
7,0.532,0.874,9,-5.183,0,0.0482,0.00842,0.012000,0.452,0.361,91.038,0U0ldCRmgCqhVvD6ksG63j
8,0.594,0.679,7,-3.702,1,0.3450,0.48400,0.000008,0.114,0.660,106.726,1dp8aQANyTRKssDeAYPiZe
...,...,...,...,...,...,...,...,...,...,...,...,...


In [531]:
#generate genres of playlist
temp_genre = genre_generate(features_df['track_id'])
temp_genre

{0: {'g funk', 'gangster rap', 'hip hop', 'rap', 'west coast rap'},
 1: {'conscious hip hop', 'hip hop', 'rap', 'west coast rap'},
 2: {'east coast hip hop', 'hip hop', 'rap'},
 3: {'atl hip hop', 'hip hop', 'rap'},
 4: {'barbadian pop',
  'conscious hip hop',
  'hip hop',
  'pop rap',
  'rap',
  'west coast rap',
  'westcoast flow'},
 5: {'alternative hip hop',
  'cali rap',
  'drumless hip hop',
  'hip hop',
  'instrumental hip hop',
  'west coast rap'},
 6: {'filter house'},
 7: {'filter house'},
 8: {'gangster rap',
  'hip hop',
  'melodic rap',
  'philly rap',
  'pop rap',
  'rage rap',
  'rap',
  'southern hip hop',
  'trap',
  'virginia hip hop'},
 9: {'hip hop', 'pop rap', 'rap', 'underground hip hop'},
 10: {'conscious hip hop',
  'hip hop',
  'rap',
  'tennessee hip hop',
  'underground hip hop',
  'west coast rap'},
 11: {'canadian hip hop',
  'canadian trap',
  'melodic rap',
  'pop rap',
  'r&b',
  'rap',
  'trap'},
 12: {'hip hop', 'melodic rap', 'philly rap', 'rage rap',

In [458]:
temp = 0
count = 0
for key in temp_genre:
    #print(key)
    if key!=temp:
        count +=1
    temp +=1
    
count

#no skipped values for first x_number values, this is our way to double check if our code had to skip a row or not

0

In [536]:
column_genre = []

for value in temp_genre.values():
    column_genre.append(list(value))
    
column_genre
print(len(column_genre))

23


In [537]:
features_df['genre'] = column_genre

features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,genre
0,0.872,0.723,1,-5.968,1,0.2470,0.02930,0.000000,0.327,0.843,93.279,4VQNCzfZ3MdHEwwErNXpBo,"[west coast rap, rap, g funk, gangster rap, hi..."
1,0.800,0.585,10,-7.343,1,0.0924,0.26400,0.000000,0.153,0.779,126.058,6PGoSes0D9eUDeeAafB2As,"[conscious hip hop, hip hop, rap, west coast rap]"
2,0.600,0.537,9,-7.777,0,0.0362,0.04680,0.003800,0.206,0.126,90.061,6Ec5LeRzkisa5KJtwLfOoW,"[rap, east coast hip hop, hip hop]"
3,0.743,0.344,1,-11.174,1,0.1180,0.16900,0.008790,0.103,0.577,160.150,0wXuerDYiBnERgIpbb3JBR,"[hip hop, atl hip hop, rap]"
4,0.775,0.736,1,-8.072,0,0.2470,0.31900,0.001160,0.127,0.362,100.988,2g6tReTlM2Akp41g0HaeXN,"[conscious hip hop, west coast rap, rap, westc..."
5,0.621,0.627,7,-7.762,0,0.1360,0.20900,0.000131,0.667,0.283,71.053,0GSbjG7c0gm7qX0Ru7vuon,"[cali rap, west coast rap, alternative hip hop..."
6,0.403,0.646,0,-8.190,0,0.0372,0.44300,0.000557,0.131,0.418,93.981,0qKX14YZHptDWiEN0CgxGz,[filter house]
7,0.532,0.874,9,-5.183,0,0.0482,0.00842,0.012000,0.452,0.361,91.038,0U0ldCRmgCqhVvD6ksG63j,[filter house]
8,0.594,0.679,7,-3.702,1,0.3450,0.48400,0.000008,0.114,0.660,106.726,1dp8aQANyTRKssDeAYPiZe,"[southern hip hop, philly rap, rage rap, rap, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [538]:
float_cols2 = features_df.dtypes[features_df.dtypes == 'float64'].index.values


feature_set_playlist = create_feature_set(features_df, float_cols=float_cols2)#.mean(axis = 0)

feature_set_playlist
#same weighted tfidf map for the playlist csv i imported

,genre|alternative,genre|atl,genre|australian,genre|barbadian,genre|cali,genre|canadian,genre|chicago,genre|coast,genre|conscious,genre|drumless,genre|east,genre|fam,genre|filter,genre|flow,genre|funk,genre|gangster,genre|hardcore,genre|hip,genre|hop,genre|house,genre|indie,genre|instrumental,genre|melodic,genre|modern,genre|neo,genre|philly,genre|pittsburgh,genre|pop,genre|psych,genre|psychedelic,genre|queens,genre|rage,genre|rap,genre|rock,genre|southern,genre|tennessee,genre|trap,genre|underground,genre|virginia,genre|weirdcore,genre|west,genre|westcoast,genre|wu,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.295912,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.485876,0.376511,0.000000,0.186547,0.186547,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.583921,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.352189,0.000000,0.00000,0.185570,0.142815,0.163000,0.121171,0.005525,0.000000,0.087193,0.200000,0.040418,4VQNCzfZ3MdHEwwErNXpBo
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.334348,0.425415,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.421556,0.421556,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.439844,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.397934,0.000000,0.00000,0.161409,0.102346,0.140939,0.034122,0.058067,0.000000,0.029463,0.184089,0.100027,6PGoSes0D9eUDeeAafB2As
2,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.382699,0.000000,0.000000,0.569673,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.482518,0.482518,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.251726,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.094295,0.088270,0.133975,0.002477,0.009443,0.014153,0.047047,0.021753,0.034566,6Ec5LeRzkisa5KJtwLfOoW
3,0.000000,0.699089,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.474359,0.474359,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.247469,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.142282,0.031672,0.079471,0.048536,0.036800,0.032737,0.012873,0.133872,0.162024,0wXuerDYiBnERgIpbb3JBR
4,0.000000,0.000000,0.0,0.351518,0.000000,0.0,0.00000,0.189176,0.240703,0.000000,0.000000,0.00000,0.000000,0.351518,0.000000,0.000000,0.000000,0.238519,0.238519,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.481405,0.0,0.0,0.000000,0.000000,0.373300,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.225154,0.351518,0.00000,0.153020,0.146628,0.129242,0.121171,0.070380,0.004320,0.020836,0.080423,0.054437,2g6tReTlM2Akp41g0HaeXN
5,0.335494,0.000000,0.0,0.000000,0.335494,0.0,0.00000,0.180553,0.000000,0.335494,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.455292,0.455292,0.000000,0.000000,0.335494,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.237522,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.214890,0.000000,0.00000,0.101342,0.114663,0.134216,0.058671,0.045754,0.000488,0.200000,0.060783,0.000000,0GSbjG7c0gm7qX0Ru7vuon
6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.028188,0.120235,0.127349,0.003041,0.098140,0.002074,0.022163,0.094344,0.041695,0qKX14YZHptDWiEN0CgxGz
7,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.0,0.0,0.00

In [539]:
# we need these dataframes to be the same length in order for our cosine similarity to work:
#lets find all listed genres and create blank columns where necessary:
total_genres = []
for entry in features_df['genre']:
    #print(entry)
    for genre in entry:
        #print(genre)
        if f'genre|{genre}' not in total_genres:
            total_genres.append(f'genre|{genre}')
            
total_genres
import ast
for entry in spotify_reduced['genre']:
    #print(list(entry))
    #x = '[ "A","B","C" , " D"]'
    x = ast.literal_eval(entry)
    #print(x)
    for genre in x:
        #print(genre)
        if f'genre|{genre}' not in total_genres:
            total_genres.append(f'genre|{genre}')
            
total_genres
print(len(total_genres))

583


In [540]:
temp = []
for entry in feature_set_playlist:
    temp.append(entry)


for entry in total_genres:
    if entry not in temp:
        feature_set_playlist[entry] = float(0)
#feature_set_playlist = feature_set_playlist.convert_dtypes()
feature_set_playlist

#added other genre columns and set to 0

,genre|alternative,genre|atl,genre|australian,genre|barbadian,genre|cali,genre|canadian,genre|chicago,genre|coast,genre|conscious,genre|drumless,genre|east,genre|fam,genre|filter,genre|flow,genre|funk,genre|gangster,genre|hardcore,genre|hip,genre|hop,genre|house,genre|indie,genre|instrumental,genre|melodic,genre|modern,genre|neo,genre|philly,genre|pittsburgh,genre|pop,genre|psych,genre|psychedelic,genre|queens,genre|rage,genre|rap,genre|rock,genre|southern,genre|tennessee,genre|trap,genre|underground,genre|virginia,genre|weirdcore,...,genre|shoegaze,genre|boston rock,genre|dream pop,genre|melodic metalcore,genre|welsh metal,genre|uk metalcore,genre|brighton indie,genre|deep ccm,genre|chilean hardcore,genre|ska chileno,genre|pop emo,genre|rap rock,genre|progressive rock,genre|progressive metal,genre|political hip hop,genre|speedrun,genre|video game music,genre|rock mineiro,genre|canadian pop,genre|uk post-punk,genre|new wave,genre|post-punk,genre|post-punk argentina,genre|candy pop,genre|protopunk,genre|latin viral pop,genre|trap latino,genre|concepcion indie,genre|chilean indie,genre|pop chileno,genre|australian electropop,genre|australian dance,genre|anime,genre|j-poprock,genre|turkish psych,genre|argentine telepop,genre|riot grrrl,genre|colombian rock,genre|reggae uruguayo,genre|rock uruguayo
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.295912,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.485876,0.376511,0.000000,0.186547,0.186547,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.583921,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.334348,0.425415,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.421556,0.421556,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.439844,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.382699,0.000000,0.000000,0.569673,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.482518,0.482518,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.251726,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.699089,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.474359,0.474359,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.247469,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.351518,0.000000,0.0,0.00000,0.189176,0.240703,0.000000,0.000000,0.00000,0.000000,0.351518,0.000000,0.000000,0.000000,0.238519,0.238519,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.481405,0.0,0.0,0.000000,0.000000,0.373300,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.335494,0.000000,0.0,0.000000,0.335494,0.0,0.00000,0.180553,0.000000,0.335494,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.455292,0.455292,0.000000,0.000000,0.335494,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.237522,0.0,0.000000,0.0,0.000

In [541]:
temp = []
for entry in complete_feature_set:
    temp.append(entry)


for entry in feature_set_playlist:
    if entry not in temp:
        complete_feature_set[entry] = float(0)
        
complete_feature_set
#added other genre columns and set to 0

,genre|acoustic,genre|acustico,genre|african,genre|afro,genre|afrobeat,genre|afrobeats,genre|afrocolombiano,genre|afrofuturismo,genre|afroperuana,genre|afropop,genre|alberta,genre|album,genre|albuquerque,genre|alt,genre|alte,genre|alternative,genre|alternativo,genre|american,genre|americana,genre|anadolu,genre|and,genre|anime,genre|ann,genre|anthem,genre|anti,genre|arab,genre|arbor,genre|area,genre|arena,genre|argentina,genre|argentine,genre|argentino,genre|art,genre|athens,genre|australian,genre|axe,genre|azonto,genre|baiano,genre|band,genre|baroque,...,genre|pop rock brasileiro,genre|birmingham metal,genre|album rock,genre|glam metal,genre|boston rock,genre|dream pop,genre|melodic metalcore,genre|welsh metal,genre|uk metalcore,genre|brighton indie,genre|deep ccm,genre|chilean hardcore,genre|ska chileno,genre|pop emo,genre|rap rock,genre|progressive rock,genre|progressive metal,genre|political hip hop,genre|video game music,genre|rock mineiro,genre|canadian pop,genre|uk post-punk,genre|new wave,genre|post-punk,genre|post-punk argentina,genre|candy pop,genre|latin viral pop,genre|trap latino,genre|concepcion indie,genre|chilean indie,genre|pop chileno,genre|australian electropop,genre|australian dance,genre|j-poprock,genre|turkish psych,genre|argentine telepop,genre|riot grrrl,genre|colombian rock,genre|reggae uruguayo,genre|rock uruguayo
0,0.257037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.385458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.176364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.335884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.277942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.376792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.224272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [542]:
temp = []
for entry in feature_set_playlist:
    temp.append(entry)
temp2 = []
for entry in complete_feature_set:
    temp2.append(entry)

for z in temp2:
    if z not in temp:
        feature_set_playlist[z] = float(0)
feature_set_playlist

#added other missed genres

,genre|alternative,genre|atl,genre|australian,genre|barbadian,genre|cali,genre|canadian,genre|chicago,genre|coast,genre|conscious,genre|drumless,genre|east,genre|fam,genre|filter,genre|flow,genre|funk,genre|gangster,genre|hardcore,genre|hip,genre|hop,genre|house,genre|indie,genre|instrumental,genre|melodic,genre|modern,genre|neo,genre|philly,genre|pittsburgh,genre|pop,genre|psych,genre|psychedelic,genre|queens,genre|rage,genre|rap,genre|rock,genre|southern,genre|tennessee,genre|trap,genre|underground,genre|virginia,genre|weirdcore,...,genre|southampton,genre|spanish,genre|speed,genre|stomp,genre|stoner,genre|swedish,genre|synth,genre|synthpop,genre|talent,genre|tech,genre|techno,genre|teen,genre|telepop,genre|texas,genre|thrash,genre|togolese,genre|tradicional,genre|traditional,genre|trip,genre|trumpet,genre|tucson,genre|tuga,genre|turkish,genre|turkuleri,genre|uk,genre|urban,genre|urbano,genre|uruguayo,genre|venezolana,genre|venezuelan,genre|veracruz,genre|vibe,genre|video,genre|violin,genre|viral,genre|vocal,genre|wave,genre|welsh,genre|youth,genre|zambian
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.295912,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.485876,0.376511,0.000000,0.186547,0.186547,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.583921,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.334348,0.425415,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.421556,0.421556,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.439844,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.382699,0.000000,0.000000,0.569673,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.482518,0.482518,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.251726,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.699089,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.474359,0.474359,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.247469,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.351518,0.000000,0.0,0.00000,0.189176,0.240703,0.000000,0.000000,0.00000,0.000000,0.351518,0.000000,0.000000,0.000000,0.238519,0.238519,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.481405,0.0,0.0,0.000000,0.000000,0.373300,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.335494,0.000000,0.0,0.000000,0.335494,0.0,0.00000,0.180553,0.000000,0.335494,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.455292,0.455292,0.000000,0.000000,0.335494,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.237522,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000

In [543]:
#order these in same way:
#run the del only if necessary, depends on order you ran your cells
#lets save the track ids then delete them
feature_set_trackid = feature_set_playlist['track_id']
complete_set_trackid = complete_feature_set['track_id']

del feature_set_playlist['track_id']
del complete_feature_set['track_id']
feature_set_playlist = feature_set_playlist.reindex(sorted(feature_set_playlist.columns), axis=1)
complete_feature_set = complete_feature_set.reindex(sorted(feature_set_playlist.columns), axis=1)

In [544]:
def generate_playlist_vector(playlist_feature):
    '''
    takes playlist feature dataframe and sums the columns to make final vector
    '''
    list_ = []
    playlist = playlist_feature
    #del playlist['track_id']
    for entry in playlist:
        list_.append(feature_set_playlist[entry].sum())
    
    res = np.array(list_)
    return res

playlist_vector = generate_playlist_vector(feature_set_playlist)
playlist_vector

#take the sum of the playlist weights

array([1.46535181, 2.35671141, 2.48152493, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.33549429, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [545]:
cosine_similar = []
for i in range(2495):
    #print(np.array(list(complete_feature_set.iloc[i])))
    A = np.array(list(complete_feature_set.iloc[i]))
    B = playlist_vector
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    cosine_similar.append(cosine)
    #print("Cosine Similarity:", cosine)
    
#get cosine similarity of each row on base dataset vs. playlist vector
#got this code from: https://www.geeksforgeeks.org/how-to-calculate-cosine-similarity-in-python/

In [546]:
complete_feature_set['cosine'] = cosine_similar
complete_feature_set['track_id'] = complete_set_trackid

complete_feature_set = complete_feature_set.sort_values(by=['cosine'])
complete_feature_set
#add cosine values to big dataframe
#sort by this column

,acousticness,danceability,energy,genre|acoustic,genre|acoustic blues,genre|acoustic chill,genre|acoustic cover,genre|acoustic guitar cover,genre|acoustic opm,genre|acoustic pop,genre|acoustic punk,genre|acoustic rock,genre|acustico,genre|african,genre|african electronic,genre|afro,genre|afro psych,genre|afro-funk,genre|afrobeat,genre|afrobeat brasileiro,genre|afrobeat fusion,genre|afrobeats,genre|afrocolombiano,genre|afrofuturismo,genre|afrofuturismo brasileiro,genre|afroperuana,genre|afropop,genre|alberta,genre|alberta country,genre|album,genre|album rock,genre|albuquerque,genre|albuquerque indie,genre|alt,genre|alt z,genre|alte,genre|alternative,genre|alternative americana,genre|alternative dance,genre|alternative hip hop,...,genre|veracruz indie,genre|vibe,genre|video,genre|video game music,genre|violao,genre|violin,genre|viral,genre|viral pop,genre|virginia,genre|virginia hip hop,genre|vocal,genre|wave,genre|weirdcore,genre|welsh,genre|welsh metal,genre|west,genre|west african jazz,genre|west coast rap,genre|westcoast,genre|westcoast flow,genre|wonky,genre|world,genre|world worship,genre|worship,genre|wu,genre|wu fam,genre|youth,genre|zambian,genre|zambian gospel,genre|zambian hip hop,genre|zambian pop,genre|zolo,instrumentalness,liveness,loudness,speechiness,tempo,valence,cosine,track_id
451,0.198185,0.043155,0.006524,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.525458e-02,0.016308,0.061176,0.002642,0.039727,0.016130,0.048260,3V0NxWbbUTa3BUAnchm11i
402,0.198185,0.078654,0.004044,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.083503e-02,0.015875,0.063011,0.004218,0.085382,0.023025,0.059857,7AKxbxzkxm2ERWrd1ggniO
262,0.173992,0.072854,0.023392,0.502449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.757637e-07,0.015957,0.113206,0.009257,0.012087,0.023234,0.060868,6Xbi3gOwMal0Ds3ETvBBwe
284,0.184677,0.144316,0.006178,0.309632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.017751,0.026019,0.007814,0.085286,0.021354,0.061902,0njuNwIyB3DoBHC0SFUkOr
606,0.053225,0.034107,0.037212,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.818737e-02,0.068055,0.110547,0.002220,0.111653,0.030338,0.063800,685WYfzmOeDOF4laU8UuOv
1,0.186290,0.071462,0.031318,0.385458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.132383e-06,0.016308,0.083388,0.011810,0.029924,0.047890,0.064005,4qPNDBW1i3p13qLCt0Ki3A
830,0.100403,0.038283,0.037821,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [549]:
#lets show the top 25 reccomendations:
complete_feature_set.head(25)

top_tracks = complete_feature_set.head(25)['track_id']

name = []
link = []
artist = []
track_ids = []
for track_id in top_tracks:
    track_ids.append(track_id)

    CLIENT_ID = "ef3a0b8293f84aae9cc15dda27c9b462"
    CLIENT_SECRET = "373610a9af5445d6925f21938ace6fc6"

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

    track_data = sp.track(track_id)
    link.append(track_data['external_urls'])
    name.append(track_data['album']['name'])

    artist_ids = []
    temp = []
    for z in track_data["artists"]:
        temp.append(z['name'])
    artist.append(temp)
        
#get the top tracks by taking the top values from previous cosine column, then get artist, name, link infor

In [7]:
# print(len(name))
# print(len(artist))
# print(len(link))

reccomendations = pd.DataFrame()
reccomendations['name'] = name
reccomendations['artist'] = artist
reccomendations['link'] = link
reccomendations['track_id'] = track_ids
reccomendations

#display information from previous cell

NameError: name 'pd' is not defined

In [ ]:
#other resources consulted:
#https://www.youtube.com/watch?v=tooddaC14q4&ab_channel=MadhavThaker 
#https://github.com/simon-th/spotify-data-project/blob/master/Extracting%20Spotify%20Audio%20Features.ipynb 
#https://www.geeksforgeeks.org/how-to-calculate-cosine-similarity-in-python/